In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("daily_IBM.csv")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers import ConvLSTM2D

dataframe = read_csv('daily_IBM.csv', usecols=[4])

In [3]:
dataset = dataframe.values
dataset = dataset.astype('float32') #Convert values to float

train_size = int(len(dataset) * 0.66)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

def to_sequences(dataset, seq_size=1):
    x = []
    y = []

    for i in range(len(dataset)-seq_size-1):
        window = dataset[i:(i+seq_size), 0]
        x.append(window)
        y.append(dataset[i+seq_size, 0])      
    return np.array(x),np.array(y)   

seq_size = 10  # Number of time steps to look back 

trainX, trainY = to_sequences(train, seq_size)
testX, testY = to_sequences(test, seq_size)

print("Shape of training set: {}".format(trainX.shape))
print("Shape of test set: {}".format(testX.shape))


#Bidirectional ConvLSTM 

from keras.layers import Bidirectional

trainX = trainX.reshape((trainX.shape[0], 1, 1, 1, seq_size))
testX = testX.reshape((testX.shape[0], 1, 1, 1, seq_size))

model = Sequential()
model.add(Bidirectional(ConvLSTM2D(filters=64, kernel_size=(1,1), activation='relu'), input_shape=(1, 1, 1, seq_size)))
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
#print('Train...')

model.fit(trainX, trainY, validation_data=(testX, testY),
          verbose=2, epochs=1000)

Shape of training set: (55, 10)
Shape of test set: (23, 10)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 1, 1, 128)         38400     
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 42,561
Trainable params: 42,561
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
2/2 - 1s - loss: 11324.2783 - val_loss: 11249.2422
Epoch 2/1000
2/2 - 0s - loss: 8712.0840 - val_loss: 8626.5029
Epoch 3/1000
2/2 - 0s - loss: 6687.2432 - val_loss: 6663.1401
Ep

In [4]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))

testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

# shift train predictions for plotting 
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[seq_size:len(trainPredict)+seq_size, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(seq_size*2)+1:len(dataset)-1, :] = testPredict

Train Score: 2.07 RMSE
Test Score: 1.94 RMSE


In [5]:
pip install pywebio

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pywebio
from pywebio.input import input, FLOAT
from pywebio.output import put_text, put_html, put_markdown, put_table

In [7]:
from numpy import array
x=[]
def stock_price():
    p1 = input("Input prices 1：", type=FLOAT)
    p2 = input("Input prices 2 ：", type=FLOAT)
    p3 = input("Input prices 3 ：", type=FLOAT)
    p4 = input("Input prices 4：", type=FLOAT)
    p5 = input("Input prices 5：", type=FLOAT)
    p6 = input("Input prices 6：", type=FLOAT)
    p7 = input("Input prices 7：", type=FLOAT)
    p8 = input("Input prices 8：", type=FLOAT)
    p9 = input("Input prices 9：", type=FLOAT)
    p10 = input("Input prices 10：", type=FLOAT)
    x = array([p1,p2,p3,p4,p5,p6,p7,p8,p9,p10])
    x = x.reshape((1, 1, 1, 1, 10))
    final = model.predict(x, verbose=0)
    put_markdown('# **Results**')
    put_text('Final results: %.1f' % (final))
    put_html('<br><br>')
    put_markdown('Final results %1f' % (final))
    put_html('<hr>')

if __name__ == '__main__':
    pywebio.start_server(stock_price, port=10) #If the link generated has any problems then try changing the port value

#a runtime error may occur while running this cell as the cell will be continously running.
#It does not affect the link generation or results.Just click on the link generated and input 10 previous days price and view the present day price priction.

Running on all addresses.
Use http://192.168.200.222:10/ to access the application


RuntimeError: This event loop is already running